# Processing Plot Lidar

In this notebook we will create a lidar point cloud for each plot. This includes
- Cropping to the plot geometry
- Loading Z field into Altitude
- Loading Height Above Ground into Z
- Filtering out labeled noise
- Saving cloud as a cloud optimised point cloud (COPC)

To process the data we will use PDAL pipelines.
We will also use dask to run the processing in parallel.

In [1]:
from pathlib import Path
import json

from jinja2 import Template
import geopandas as gpd
import pdal
import pandas as pd

## Pipeline Template

In [2]:
# Note, we could have this just a string, but as a dict allows us to add comments
pipeline_template_dict = [
    # Read the input LAS file
    {
        "type": "readers.copc",
        "filename": "{{ input_path }}",
        "polygon": "{{ polygon_wkt }}",

    },
    # Only take unclassified, ground and vegetation points
    {
        "type": "filters.range",
        "limits": "Classification[0:5]",
    },
    # Load Z into Altitude and HeightAboveGround into Z
    {
        "type": "filters.ferry",
        "dimensions": "Z => Altitude, HeightAboveGround => Z"
    },
    # Classify points below the ground as noise
    {
        "type": "filters.assign",
        "value": ["Classification = 2 WHERE Z < 0", "Z = 0 WHERE Z < 0"],
    },
    # Add the weight field (1 / Number of Returns)
    {
        "type": "filters.assign",
        "value": "Weight = 1 / NumberOfReturns"
    },
    # Save as a COPC file
    {
        "type": "writers.copc",
        "filename": "{{ output_path }}",
        "forward": "scale,offset",
        "extra_dims": "all"
    }
]

pipeline_template = json.dumps(pipeline_template_dict, indent=2)

# Function to replace variables
def replace_pipeline_variables(pipeline_template: str, context: dict):
    template = Template(pipeline_template)
    return template.render(context)

### Plot pipelines

In [3]:
import geopandas as gpd

plots_gdf = gpd.read_file("../data/outputs/plots/plots.geojson")
plots_gdf.head()

,site,plot_number,site_plot_id,id,geometry
0,AGG_O_01,1,AGG_O_01_P1,AGG_O_01_P1,"POLYGON ((463042.83 5259846.736, 463025.797 52..."
1,AGG_O_01,2,AGG_O_01_P2,AGG_O_01_P2,"POLYGON ((463124.556 5259819.234, 463116.068 5..."
2,AGG_O_01,3,AGG_O_01_P3,AGG_O_01_P3,"POLYGON ((463201.174 5259815.806, 463200.551 5..."
3,AGG_O_01,4,AGG_O_01_P4,AGG_O_01_P4,"POLYGON ((463257.777 5259801.962, 463245.303 5..."
4,AGG_O_01,5,AGG_O_01_P5,AGG_O_01_P5,"POLYGON ((463303.022 5259789.552, 463289.794 5..."


In [4]:
outputs_dir = Path("../data/outputs")
sites_lidar_dir = outputs_dir / "sites" / "lidar"
plots_lidar_dir = outputs_dir / "plots" / "lidar"
plots_lidar_dir.mkdir(parents=True, exist_ok=True)

def create_pipeline_from_plot(plot_row):
    site_id = plot_row['site']
    site_plot_id = plot_row['site_plot_id']

    context = {
        "input_path": str(sites_lidar_dir / f"{site_id}.copc.laz"),
        "output_path": str(plots_lidar_dir / f"{site_plot_id}.copc.laz"),
        "polygon_wkt": plot_row.geometry.wkt
    }
    return replace_pipeline_variables(pipeline_template, context)

pipelines = plots_gdf.apply(create_pipeline_from_plot, axis=1).to_list()
print(pipelines[0])

[
  {
    "type": "readers.copc",
    "filename": "../data/outputs/sites/lidar/AGG_O_01.copc.laz",
    "polygon": "POLYGON ((463042.83002541395 5259846.735807601, 463025.79692534194 5259799.726515798, 462975.6639067796 5259817.891450633, 462992.69700685085 5259864.900742435, 463042.83002541395 5259846.735807601))"
  },
  {
    "type": "filters.range",
    "limits": "Classification[0:5]"
  },
  {
    "type": "filters.ferry",
    "dimensions": "Z => Altitude, HeightAboveGround => Z"
  },
  {
    "type": "filters.assign",
    "value": [
      "Classification = 2 WHERE Z < 0",
      "Z = 0 WHERE Z < 0"
    ]
  },
  {
    "type": "filters.assign",
    "value": "Weight = 1 / NumberOfReturns"
  },
  {
    "type": "writers.copc",
    "filename": "../data/outputs/plots/lidar/AGG_O_01_P1.copc.laz",
    "forward": "scale,offset",
    "extra_dims": "all"
  }
]


## Processing

In [5]:
def process_pdal_pipeline(pipeline: str, return_data: bool = False):
    """
    Process a PDAL pipeline string.

    Args:
        pipeline (str): The PDAL pipeline JSON string.
        return_data (bool): If True, return the PDAL Pipeline object after execution. Defaults to False. Returning pipeline data
        will contain metadata and all the points processed by the pipeline. This can be a large object so defaults to False.
    """
    pipeline_obj = pdal.Pipeline(pipeline)
    count = pipeline_obj.execute()  # Execute the pipeline
    return (count, pipeline_obj if return_data else None)

In [6]:
%%time

(count, pl) = process_pdal_pipeline(pipelines[0], return_data=True)
print(f"Processed {count} points.")

points = pl.arrays[0]
points_df = pd.DataFrame(pl.arrays[0])
points_df.head()

Processed 252716 points.
CPU times: user 947 ms, sys: 42.5 ms, total: 990 ms
Wall time: 737 ms


,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,Synthetic,...,PointSourceId,GpsTime,ScanChannel,Red,Green,Blue,Infrared,HeightAboveGround,Altitude,Weight
0,463020.976,5259853.864,9.702999,30729,1,2,0,0,0,0,...,1,4.118795e+08,0,38036,36237,34695,46277,9.702999,515.146,0.500000
1,463021.434,5259854.039,0.000000,28635,3,3,0,0,2,0,...,1,4.118795e+08,0,26471,24929,24672,41676,0.000000,505.447,0.333333
2,463020.911,5259851.998,0.000000,29477,2,2,0,0,2,0,...,1,4.118795e+08,0,22873,23387,25957,33477,0.000000,505.373,0.500000
3,463021.056,5259852.351,0.000000,28471,3,3,0,0,2,0,...,1,4.118795e+08,0,19532,19532,22102,32570,0.000000,505.413,0.333333
4,463021.130,5259852.531,1.196000,28481,4,4,0,0,0,0,...,1,4.118795e+08,0,31354,31097,31868,29393,1.196000,506.684,0.250000


In [7]:
from dask.distributed import Client

client = Client()  # Start a Dask client
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54238,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:54250,Total threads: 2
Dashboard: http://127.0.0.1:54253/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:54241,


In [8]:
futures = client.map(process_pdal_pipeline, pipelines, key=plots_gdf["site_plot_id"].tolist())

In [9]:
client.gather(futures)

[(252716, None),
 (224263, None),
 (275076, None),
 (286625, None),
 (282157, None),
 (140941, None),
 (141178, None),
 (133969, None),
 (154271, None),
 (163370, None),
 (156898, None),
 (125439, None),
 (123924, None),
 (140339, None),
 (130446, None),
 (122761, None),
 (115035, None),
 (104534, None),
 (206294, None),
 (187184, None),
 (180367, None),
 (199822, None),
 (125271, None),
 (77728, None),
 (111543, None),
 (112151, None),
 (156136, None),
 (199051, None),
 (151712, None),
 (175477, None),
 (187612, None),
 (161808, None),
 (133269, None),
 (155056, None),
 (171943, None),
 (175053, None),
 (162860, None),
 (134646, None),
 (104395, None),
 (111741, None),
 (228261, None),
 (231726, None),
 (223968, None),
 (285092, None),
 (251251, None),
 (218543, None),
 (208965, None),
 (158307, None),
 (129117, None),
 (119538, None),
 (91654, None),
 (112863, None),
 (105605, None),
 (117650, None),
 (107662, None),
 (257093, None),
 (251103, None),
 (329963, None),
 (308931, None),

In [10]:
client.close()